In [1]:
pip install ipynb --upgrade

Requirement already up-to-date: ipynb in /Users/mattiaskallman1/anaconda3/lib/python3.7/site-packages (0.5.1)
Note: you may need to restart the kernel to use updated packages.


In [22]:
import pickle
import time
import os
import yaml
import sys
import pandas as pd

os.chdir("..")
import Models
os.chdir("MockPipeline")

os.chdir("../../pipeline")
import ipynb.fs.full.processing as processing
import ipynb.fs.full.features as features
import ipynb.fs.full.training as training
import ipynb.fs.full.analysis as analysis
import ipynb.fs.full.storage as storage
import ipynb.fs.full.visualize as visualize
os.chdir("../models-decision-system/MockPipeline")

from tensorflow.keras.layers import Dense, Dropout
from tensorflow.python.keras import Input, Model

from tcn import TCN, tcn_full_summary

In [3]:
os.getcwd()


'/Users/mattiaskallman1/Documents/Arcada/2020-2021/Decision_support_system/models-decision-system/MockPipeline'

In [4]:
def load_yaml(path):
    with open(path, mode='r') as file:
        return yaml.load(file, Loader=yaml.FullLoader)

In [5]:
settings = load_yaml('mock_settings.yaml')

In [6]:
#settings['ensemble']['models'][0]

In [7]:
dataframe = processing.create_dataframe(settings)

In [8]:
dataset = features.add(dataframe, settings['features'])

In [9]:
#dataset

In [10]:
data_linreg = features.split(dataset, 'linreg', settings['ensemble']['models'][0]['linreg'])

In [11]:
# WORKING
#linreg = Models.train_model(data_linreg, 'linreg', settings['ensemble']['models'][0])

In [12]:
data_lstm = features.split(dataset, 'lstm', settings['ensemble']['models'][1]['lstm'])

In [13]:
#settings['ensemble']['models'][1]['lstm']['layers']

[{'lstm': {'value': 120, 'activation': 'relu'}},
 {'dropout': {'value': 0.15}},
 {'dense': {'value': 50, 'activation': 'relu'}},
 {'dense': {'value': 1}}]

In [14]:
# WORKING
#lstm = Models.train_model(data_lstm, 'lstm', settings['ensemble']['models'][1]['lstm'])

In [17]:
def add_base_layer_to_tcn(name, model_output, settings):
    available = {
        'dropout': Dropout,
        'dense': Dense
    }
    func = available[name]
    
    if name == 'dropout':
        return func(settings['value'])(model_output)
    else:
        return func(settings['value'])(model_output)
    
def add_tcn_layers(model_input, settings):
    model_output = TCN(return_sequences=False)(model_input)
    for index, layer in enumerate(settings['layers']):
        name = list(layer)[0]
        params = layer[name]
        if index == 0:
            continue
        else:
            model_output = add_base_layer_to_tcn(name, model_output, params)
    return model_output

In [19]:
def temporal_convolutional_network(data, settings):

    #  VARIABLES
    x_train = data['train']['features']
    y_train = data['train']['labels']
    x_validation = data['validation']['features']
    y_validation = data['validation']['labels']
    x_test = data['test']['features']
    scaler = data['scaler']
    timesteps = x_train.shape[0]
    input_dim = x_train.shape[1]
    batch_size = settings['batch']

    model_input = Input(shape=(input_dim, 1))

    model_output = add_tcn_layers(model_input, settings)
    
    model = Model(inputs=[model_input], outputs=[model_output])
    
    model.compile(optimizer='adam', loss='mse')

    tcn_full_summary(model, expand_residual_blocks=False)
    
    model.fit(x_train, y_train, epochs=settings['epochs'], validation_data=(x_validation, y_validation))
    #  https://keras.io/api/models/model_training_apis/#fit-method

    predictions = model.predict(x_test)

    return {
        'model': model,
        'predictions': predictions
    }

In [20]:
#temporal_convolutional_network(data_lstm, settings['ensemble']['models'][3]['tcn'])

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 3, 1)]            0         
_________________________________________________________________
residual_block_0 (ResidualBl [(None, 3, 64), (None, 3, 8576      
_________________________________________________________________
residual_block_1 (ResidualBl [(None, 3, 64), (None, 3, 16512     
_________________________________________________________________
residual_block_2 (ResidualBl [(None, 3, 64), (None, 3, 16512     
_________________________________________________________________
residual_block_3 (ResidualBl [(None, 3, 64), (None, 3, 16512     
_________________________________________________________________
residual_block_4 (ResidualBl [(None, 3, 64), (None, 3, 16512     
_________________________________________________________________
residual_block_5 (ResidualBl [(None, 3, 64), (None, 3, 1651

{'model': <tensorflow.python.keras.engine.training.Model at 0x7fa959940cd0>,
 'predictions': array([[0.33259094],
        [0.31801304],
        [0.31172094],
        [0.3194028 ],
        [0.34248632],
        [0.3712119 ],
        [0.37540042],
        [0.3949576 ],
        [0.4008281 ],
        [0.38385698],
        [0.39911747],
        [0.39337125],
        [0.40540388],
        [0.40374675],
        [0.41468993],
        [0.42249042],
        [0.41495925],
        [0.42250818],
        [0.43243557],
        [0.4261192 ],
        [0.43557277],
        [0.43800977],
        [0.4581082 ],
        [0.44686648],
        [0.44485936],
        [0.42845854],
        [0.43252045],
        [0.4113681 ],
        [0.41421083],
        [0.41288644],
        [0.4087069 ],
        [0.4172811 ],
        [0.4283855 ],
        [0.4449547 ],
        [0.4527075 ],
        [0.43500486],
        [0.44320512],
        [0.4368869 ],
        [0.42593342],
        [0.41279152],
        [0.42831448],
      

In [21]:
tcn = Models.train_model(data_lstm, 'tcn', settings['ensemble']['models'][3]['tcn'])

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 3, 1)]            0         
_________________________________________________________________
residual_block_0 (ResidualBl [(None, 3, 64), (None, 3, 8576      
_________________________________________________________________
residual_block_1 (ResidualBl [(None, 3, 64), (None, 3, 16512     
_________________________________________________________________
residual_block_2 (ResidualBl [(None, 3, 64), (None, 3, 16512     
_________________________________________________________________
residual_block_3 (ResidualBl [(None, 3, 64), (None, 3, 16512     
_________________________________________________________________
residual_block_4 (ResidualBl [(None, 3, 64), (None, 3, 16512     
_________________________________________________________________
residual_block_5 (ResidualBl [(None, 3, 64), (None, 3, 1651